In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
# Version Information
# tensorflow 2.2.0 , Cudnn7.6.5 and Cuda 10.1 , python 3.8

In [3]:
tf.__version__

'2.4.1'

In [4]:
tf.test.is_built_with_cuda()

False

In [5]:
# nvcc --version

In [6]:
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

In [7]:
X_train.shape

(50000, 32, 32, 3)

In [8]:
y_train.shape

(50000, 1)

In [9]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [10]:
classes[y_train[3][0]]

'deer'

In [11]:
y_train[:3]

array([[6],
       [9],
       [9]], dtype=uint8)

In [12]:
y_test.shape


(10000, 1)

In [13]:
X_train.shape

(50000, 32, 32, 3)

In [14]:
# Preprocessing: Scale images
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [15]:
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=10, dtype='float32'
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=10, dtype='float32'
)

In [16]:
y_train[0:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [17]:
y_train_categorical[0:5]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [18]:
# Model building and training
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')    
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [19]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

2023-06-12 15:33:00.026395: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-06-12 15:33:00.026969: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 15:33:00.028229: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [20]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [22]:
strategy.num_replicas_in_sync

1

In [23]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [24]:
%%timeit -n1 -r1 
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset, epochs=50)

2023-06-12 15:33:08.082111: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_DOUBLE
      type: DT_FLOAT
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 10
        }
      }
    }
  }
}

2023-06-12 15:33:08.259618: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-06-12 15:33:08.278820: I tensorflow/core/platform/profile_utils/cpu_utils

Epoch 1/50
200/200 [==============================] - 35s 162ms/step - loss: 2.0822 - accuracy: 0.2509
Epoch 2/50
200/200 [==============================] - 33s 163ms/step - loss: 1.8331 - accuracy: 0.3581
Epoch 3/50
200/200 [==============================] - 33s 165ms/step - loss: 1.7619 - accuracy: 0.3872
Epoch 4/50
200/200 [==============================] - 32s 161ms/step - loss: 1.7153 - accuracy: 0.4036
Epoch 5/50
200/200 [==============================] - 33s 166ms/step - loss: 1.6786 - accuracy: 0.4152
Epoch 6/50
200/200 [==============================] - 35s 177ms/step - loss: 1.6478 - accuracy: 0.4258
Epoch 7/50
200/200 [==============================] - 37s 186ms/step - loss: 1.6210 - accuracy: 0.4344
Epoch 8/50
200/200 [==============================] - 38s 189ms/step - loss: 1.5976 - accuracy: 0.4439
Epoch 9/50
200/200 [==============================] - 35s 175ms/step - loss: 1.5768 - accuracy: 0.4515
Epoch 10/50
200/200 [==============================] - 35s 175ms/step - l

In [ ]:
%%timeit -n1 -r1 
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
200/200 [==============================] - 34s 169ms/step - loss: 2.0870 - accuracy: 0.2428
Epoch 2/50
200/200 [==============================] - 35s 176ms/step - loss: 1.8299 - accuracy: 0.3581
Epoch 3/50
200/200 [==============================] - 36s 178ms/step - loss: 1.7602 - accuracy: 0.3856
Epoch 4/50
200/200 [==============================] - 35s 174ms/step - loss: 1.7139 - accuracy: 0.4022
Epoch 5/50
200/200 [==============================] - 35s 177ms/step - loss: 1.6776 - accuracy: 0.4151
Epoch 6/50
200/200 [==============================] - 36s 178ms/step - loss: 1.6472 - accuracy: 0.4253
Epoch 7/50
200/200 [==============================] - 42s 209ms/step - loss: 1.6209 - accuracy: 0.4356
Epoch 8/50
200/200 [==============================] - 42s 212ms/step - loss: 1.5978 - accuracy: 0.4441
Epoch 9/50
200/200 [==============================] - 41s 207ms/step - loss: 1.5770 - accuracy: 0.4511
Epoch 10/50
200/200 [==============================] - 41s 205ms/step - l